In [1]:
import torch
import time
from tt_utils import *
from fcn_approx_utils import GMM, NeuralNetwork

In [2]:
device = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Fit NN-model

In [12]:
dim_list = [5,10,20]
mix_list = [5,10,20]
s_list =[0.15,0.3,0.45]

Ndim = len(dim_list)
Nmix = len(mix_list)
Ns = len(s_list)
Nt = 5
nn_gmm_data_err = torch.empty((Ndim, Nmix, Ns,Nt))
nn_gmm_data_time = torch.empty((Ndim, Nmix, Ns,Nt))


dim_list = [5,10,20]
mix_list = [10,20,40]
s_list = [0.15,0.3,0.45]

tt_gmm_data_err = torch.empty((Ndim, Nmix, Ns,Nt))
tt_gmm_data_rank = torch.empty((Ndim, Nmix, Ns,Nt))
tt_gmm_data_time = torch.empty((Ndim, Nmix, Ns,Nt))
L=1.0
for i, dim_ in enumerate(dim_list):
    for j, nmix_ in enumerate(mix_list):
        for k, s_ in enumerate(s_list):
            for p in range(Nt):
                print("###########")
                print(i,j,k,p)
                print(dim_,nmix_,s_,p)
                print("###########")
                gmm = GMM(n=dim_,nmix=nmix_,L=L,mx_coef=None,mu=None,s=s_, device=device)
                pdf = gmm.pdf
                ndata_train = 100000*dim_
                x_train = 2*L*(-0.5 + torch.rand((ndata_train,dim_)).to(device))
                y_train = pdf(x_train).view(-1,1)
                ndata_test = 10000
                x_test = 2*L*(-0.5 + torch.rand((ndata_test,dim_)).to(device))
                y_test = pdf(x_test)
                nn = NeuralNetwork(dim=dim_, width=64, lr=1e-3)
                data_train = torch.cat((x_train.view(-1,dim_),y_train.view(-1,1)),dim=-1)
                data_test = torch.cat((x_test.view(-1,dim_),y_test.view(-1,1)),dim=-1)
                nn.load_data(data_train, data_test)
                t1_nn = time.time()
                nn.train(num_epochs=10, batch_size=128, verbose=True)
                t2_nn = time.time()
                dt = t2_nn - t1_nn
                # Test the accuracy of NN over the test set
                y_nn = nn.model(x_test)
                mse_nn =  ((y_nn.view(-1)-y_test.view(-1))**2).mean().detach().cpu() #(((y_nn.view(-1)-y_test.view(-1))/(1e-9+y_test.view(-1).abs()))**2).mean().detach()
                print("mse_nn: ", mse_nn)
                nn_gmm_data_err[i,j,k,p] = 1*mse_nn
                nn_gmm_data_time[i,j,k,p] = dt
                
                n_discretization = torch.tensor([200]*dim_).to(device)
                domain = [torch.linspace(-L,L,n_discretization[i_]).to(device) for i_ in range(dim_)] 
                t1 = time.time()
                tt_gmm = cross_approximate(fcn=pdf,  max_batch=10**6, domain=domain, 
                                        rmax=200, nswp=20, eps=1e-3, verbose=False, 
                                        kickrank=10, device=device)
                t2 = time.time()

                y_tt =  get_value(tt_model=tt_gmm, x=x_test.to(device),  domain=domain, 
                                    n_discretization=n_discretization , max_batch=10**5, device=device)

                mse_tt = ((y_tt.view(-1)-y_test.view(-1))**2).mean()
                print("mse_tt: ", mse_tt)

                tt_gmm_data_err[i,j,k,p] = 1*mse_tt
                tt_gmm_data_rank[i,j,k,p] = max(tt_gmm.ranks_tt)
                tt_gmm_data_time[i,j,k,p] = (t2-t1)


                torch.save({'dim_list':dim_list,'s_list':s_list,'mix_list':mix_list,
                          'tt_gmm_data_err':tt_gmm_data_err,
                         'tt_gmm_data_time':tt_gmm_data_time,
                         'tt_gmm_data_rank':tt_gmm_data_rank},'tt_gmm_data_2.pt')
                torch.save({'dim_list':dim_list,'s_list':s_list,'mix_list':mix_list,
                          'nn_gmm_data_err':nn_gmm_data_err,
                         'nn_gmm_data_time':nn_gmm_data_time},'nn_gmm_data_2.pt')

###########
0 0 0 0
5 10 0.15 0
###########
epoch:0, loss-train:0.3965865758698981, loss-test:0.22346085023213863
epoch:1, loss-train:0.1394508428376031, loss-test:0.1012661569009718
epoch:2, loss-train:0.06086309248806345, loss-test:0.05200733891734834
epoch:3, loss-train:0.03142934088500528, loss-test:0.031096477973740412
epoch:4, loss-train:0.0186076657420402, loss-test:0.019018822611463905
epoch:5, loss-train:0.012786946734054689, loss-test:0.0121428192559925
epoch:6, loss-train:0.009896225318527628, loss-test:0.010635015936825592
epoch:7, loss-train:0.007688352458116389, loss-test:0.009205124480929796
epoch:8, loss-train:0.006570401351072717, loss-test:0.009467905186398175
epoch:9, loss-train:0.005949476294628944, loss-test:0.00837606830113768
mse_nn:  tensor(0.0084, grad_fn=<MeanBackward0>)
mse_nn:  tensor(0.0084, grad_fn=<MeanBackward0>)
###########
0 0 0 1
5 10 0.15 1
###########
epoch:0, loss-train:0.4169412110262063, loss-test:0.05088763048450542
epoch:1, loss-train:0.1241975

In [ ]:
tt_data = torch.load('tt_gmm_data_1.pt')
nn_data = torch.load('nn_gmm_data_1.pt')

In [ ]:
tt_mean = tt_data['tt_gmm_data_err'].mean(dim=(-1,-2))
tt_std = tt_data['tt_gmm_data_err'].std(dim=(-1,-2))

nn_mean = nn_data['nn_gmm_data_err'].mean(dim=(-1,-2))
nn_std = nn_data['nn_gmm_data_err'].std(dim=(-1,-2))

In [ ]:
tt_nn = (tt_data['tt_gmm_data_err']/nn_data['nn_gmm_data_err']).detach().cpu()

In [ ]:
y_mean = tt_nn.mean(dim=(-1,-2))
y_std = tt_nn.std(dim=(-1,-2))

In [ ]:
dim_list = tt_data['dim_list']
dim_list

In [ ]:
mix_list = tt_data['mix_list']
mix_list

In [ ]:
s_list = tt_data['s_list']
s_list

In [ ]:
y_mean.shape

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 10})

# Define your data
conditions = ['k=5','k=10', 'k=20']
m1_means =  y_mean[0] # Mean values for Method 1
m1_stdevs = y_std[0]   # Standard deviations for Method 1
m2_means = y_mean[1]   # Mean values for Method 2
m2_stdevs =  y_std[1]   # Standard deviations for Method 2
m3_means = y_mean[2]   # Mean values for Method 2
m3_stdevs =  y_std[2]   # Standard deviations for Method 2
m4_means = y_mean[3]   # Mean values for Method 2
m4_stdevs = y_std[3]   # Standard deviations for Method 2



# Set the width of the bars
bar_width = 0.1

# Set the positions of the bars on the x-axis
r1 = np.arange(len(conditions))-0.1
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Plot the bars
plt.bar(r1, m1_means,  width=bar_width, label='d=10', capsize=5)
plt.bar(r2, m2_means,  width=bar_width, label='d=20', capsize=5)
plt.bar(r3, m3_means,  width=bar_width, label='d=30', capsize=5)
plt.bar(r4, m4_means,  width=bar_width, label='d=40', capsize=5)

# Add labels, title, and legend
# plt.xlabel('number of mixture components',fontsize='12')
plt.ylabel('Error ratio TT/NN',fontsize='13')
# plt.title('TT vs NN')
plt.xticks([r + bar_width/2 for r in range(len(conditions))], conditions, fontsize=12)
plt.legend(ncol=4)
plt.yticks(fontsize=12)
plt.yscale('log')
plt.savefig('tt_vs_nn.jpeg', bbox_inches='tight',pad_inches=0.01, dpi=1000)
# Show the plot
plt.show()
